In [19]:
import numpy as np
import pathlib
from nilearn.decomposition import CanICA

DATADIR = pathlib.Path('/data/origami/niusha/code/local-experiment/io/ICAs')
n_components = 30

def ICA_decomposition(filenames, group, i, path=DATADIR, n=n_components):
    """
    This function receives nifiti images and calculates 30 independent components

    inputs: 
        filenames: list of input filenames or a 4D image containing all inputs concatenated.
        group: a group of subjects (PD/Healthy/Pooled). It is used for naming IC files.
        path: parent directory for storing ICs.
        i: iteration number is used for naming IC files.
        n: number of ICs.

    outputs: 
        ICA_s: A 4D image that contains 30 ICs.
    """
    fast_ica = CanICA(n_components=n,
                    memory="nilearn_cache", memory_level=2,
                    mask_strategy='whole-brain-template',
                    do_cca=False,
                    random_state=0)
    fast_ica.fit(filenames)

    ICA_s = fast_ica.components_img_
    ICA_s.to_filename(path / (f'ICAs_{group}_{i}.nii.gz'))
    return ICA_s

In [23]:
from nilearn import masking
from nilearn.image import iter_img
from nilearn.image import index_img
from nilearn.masking import apply_mask

def Means_after_masking(ICAs,DBM_maps,n=n_components):
    """
    This function,first convet each IC to a mask and then apply each mask on each subject.
    Afterthat, extracting regions of intrests, it caculate mean value of these regions.
      
    inputs:
        ICAs: IC components
        DBM_maps: Target cohort
    outputs:
        means_after_mask: an array contains mean value of each input DBM after applying IC masks
    """
    size = DBM_maps.shape[3]
    means_after_mask = np.zeros((n,size))
    for i, cur_img in enumerate(iter_img(ICAs)):
        mask_img = masking.compute_epi_mask(cur_img)
        for j in range(size):
            means_after_mask[i,j] = np.mean(apply_mask(index_img(DBM_maps,j),mask_img))
        
    return means_after_mask

In [14]:
import pandas as pd

subject_df = pd.read_csv("/data/origami/niusha/input/subject_IDs.csv")

N = len(subject_df.ID)
ID_map = dict(zip(range(N),subject_df.ID))

Healthy_index = np.where(subject_df.PD == 0)
Healthy_subject = subject_df.ID.iloc[Healthy_index]

PD_index = np.where(subject_df.PD == 1)
PD_subject = subject_df.ID.iloc[PD_index]

In [15]:
from nilearn.image import load_img
Original_DBMs = load_img("/data/origami/niusha/out/DBM_data.nii")

In [5]:
from random import sample
from nilearn.image import index_img
from scipy.stats import ttest_ind

P_vals = np.zeros((10,30))
T_vals = np.zeros((10,30))

for j in range(10):

    PD_sample = sample((PD_index[0]).tolist(), len(PD_index[0]))
    Healthy_sample = sample((Healthy_index[0]).tolist(), len(Healthy_index[0]))

    PD_bootstrapped_cohort = index_img(Original_DBMs, PD_sample)
    Healthy_bootstrapped_cohort = index_img(Original_DBMs, Healthy_sample)

    PD_ICAs = ICA_decomposition(PD_bootstrapped_cohort, "PD", j)
    Healthy_ICAs = ICA_decomposition(Healthy_bootstrapped_cohort, "Healthy", j)

    PD_means = Means_after_masking(PD_ICAs,PD_bootstrapped_cohort)
    Healthy_means = Means_after_masking(Healthy_ICAs,Healthy_bootstrapped_cohort)

    for i in range(30):
        T_vals[j,i], P_vals[j,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/joblib/memory.py:632: UserWarning: Cannot inspect object functools.partial(<function compute_multi_brain_mask at 0x7ff6b3795b20>, mask_type='whole-brain'), ignore list will not work.
  return hashing.hash(filter_args(self.func, self.ignore, args, kwargs),
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/nilearn/maskers/multi_nifti_masker.py:258: JobLibCollisionWarning: Cannot detect name collisions for function 'unknown'
  self.mask_img_ = self._cache(
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/nilearn/masking.py:568: UserWarning: Persisting input arguments took 3.47s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they c

In [7]:
# P_vals_reshaped = P_vals.reshape(P_vals.shape[0], -1)
np.savetxt("/data/origami/niusha/code/local-experiment/io/p_values.txt", P_vals)

# T_vals_reshaped = T_vals.reshape(T_vals.shape[0], -1)
np.savetxt("/data/origami/niusha/code/local-experiment/io/t_stat.txt", T_vals)

In [7]:
T_vals = np.loadtxt("/data/origami/niusha/code/local-experiment/io/t_stat.txt")

In [ ]:
T_max = np.nanmax(T_vals,axis=0)

In [ ]:
P_vals_original = np.zeros((1,30))
T_vals_original = np.zeros((1,30))
PD_cohort = index_img(Original_DBMs, (PD_index[0]).tolist())
Healthy_cohort = index_img(Original_DBMs, (Healthy_index[0]).tolist())

PD_ICAs = ICA_decomposition(PD_cohort, "PD", "")
Healthy_ICAs = ICA_decomposition(Healthy_cohort, "Healthy", "")

PD_means = Means_after_masking(PD_ICAs,PD_cohort)
Healthy_means = Means_after_masking(Healthy_ICAs,Healthy_cohort)

for i in range(30):
    T_vals_original[0,i], P_vals_original[0,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

In [11]:
np.savetxt("/data/origami/niusha/code/local-experiment/io/p_values_original.txt", P_vals_original)
np.savetxt("/data/origami/niusha/code/local-experiment/io/t_stat_original.txt", T_vals_original)

In [14]:
P_vals_whole = np.zeros((10,30))
T_vals_whole = np.zeros((10,30))

for j in range(10):

    PD_sample = sample((PD_index[0]).tolist(), len(PD_index[0]))
    Healthy_sample = sample((Healthy_index[0]).tolist(), len(Healthy_index[0]))

    PD_bootstrapped_cohort = index_img(Original_DBMs, PD_sample)
    Healthy_bootstrapped_cohort = index_img(Original_DBMs, Healthy_sample)
    Whole_bootstrapped_cohort = index_img(Original_DBMs, PD_sample + Healthy_sample)

    whole_ICAs = ICA_decomposition(Whole_bootstrapped_cohort, "whole", j)

    PD_means = Means_after_masking(whole_ICAs,PD_bootstrapped_cohort)
    Healthy_means = Means_after_masking(whole_ICAs,Healthy_bootstrapped_cohort)

    for i in range(30):
        T_vals_whole[j,i], P_vals_whole[j,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/joblib/memory.py:632: UserWarning: Cannot inspect object functools.partial(<function compute_multi_brain_mask at 0x7ff6b3795b20>, mask_type='whole-brain'), ignore list will not work.
  return hashing.hash(filter_args(self.func, self.ignore, args, kwargs),
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/nilearn/masking.py:568: UserWarning: Persisting input arguments took 5.15s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  resampled_template = cache(resampling.resample_to_img, memory)(
/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/joblib/memory.py:810: UserWarning: Cannot inspect object functools

In [15]:
np.savetxt("/data/origami/niusha/code/local-experiment/io/p_values_whole.txt", P_vals_whole)
np.savetxt("/data/origami/niusha/code/local-experiment/io/t_stat_whole.txt", T_vals_whole)

In [24]:
P_vals_whole_original = np.zeros((1,30))
T_vals_whole_original = np.zeros((1,30))


PD_sample = (PD_index[0]).tolist()
Healthy_sample = (Healthy_index[0]).tolist()

PD_bootstrapped_cohort = index_img(Original_DBMs, PD_sample)
Healthy_bootstrapped_cohort = index_img(Original_DBMs, Healthy_sample)
Whole_bootstrapped_cohort = index_img(Original_DBMs, PD_sample + Healthy_sample)

whole_ICAs = ICA_decomposition(Whole_bootstrapped_cohort, "whole", "original")

PD_means = Means_after_masking(whole_ICAs,PD_bootstrapped_cohort)
Healthy_means = Means_after_masking(whole_ICAs,Healthy_bootstrapped_cohort)

for i in range(30):
    T_vals_whole_original[0,i], P_vals_whole_original[0,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

/home/bic/niusham/.conda/envs/py10/lib/python3.11/site-packages/numpy/core/_methods.py:181: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [25]:
np.savetxt("/data/origami/niusha/code/local-experiment/io/p_values_whole_original.txt", P_vals_whole_original)
np.savetxt("/data/origami/niusha/code/local-experiment/io/t_stat_whole_original.txt", T_vals_whole_original)

In [26]:
P_vals_whole_original_loop_test =  np.loadtxt("/data/origami/niusha/code/local-experiment/io/p_values_whole_original_loop_test.txt")
P_vals_whole_original = np.loadtxt("/data/origami/niusha/code/local-experiment/io/p_values_whole_original.txt")

In [27]:
P_vals_whole_original_loop_test == P_vals_whole_original

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [ ]:
# from nilearn.plotting import plot_prob_atlas

# # Plot all ICA components together
# plot_prob_atlas(canica_components_img, title='All ICA components')

# from nilearn.image import iter_img
# from nilearn.plotting import plot_stat_map, show

# for i, cur_img in enumerate(iter_img(canica_components_img)):
#     plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
#                   cut_coords=1, colorbar=False)

In [ ]:
# from nilearn.glm import threshold_stats_img

# ICA_tr = threshold_stats_img(stat_img=ICAs, threshold=3.0)
#The output is not compatible with mask and also I dont know what is the ouput of ICA. I tought it would be z stat, but when I set thr=3 I receive warning regarding the amount